In [1]:
import os
import cv2
import numpy as np
import face_recognition
import concurrent.futures
import multiprocessing

### This notebook grab the video and extract the frames having faces only

- The face_preprocessing function takes the videos from input_path and saves the frames in Output path 
- It creates filename same as video name and creates two sub folders in it namely face and flow .
- It only populates the face folder 

- The expected directory structure for the function is shown below

<img src='directory_flow1.png'>

- The expected output directory structure
<img src='directory_flow2.png'>



In [2]:
os.getcwd()

'/home/sreejan774/Desktop/projects/MinorCodes'

In [3]:
def extract_frames(video,output_path):
    video_name = video.split('/')[-1]
    os.mkdir(os.path.join(output_path + '/'+ video_name))
    outputPath = os.path.join(output_path + '/'+ video_name)
    #print(outputPath)
    os.mkdir(os.path.join(outputPath+'/face'))
    os.mkdir(os.path.join(outputPath+'/flow'))
    facePath = os.path.join(outputPath+'/face')
    flowPath = os.path.join(outputPath+'/flow')


    # Extracting face cropped video
    cap = cv2.VideoCapture(video)


    # read frames until a face is encountered
    # if no face is encountered then return
    face_image = None
    while(True):
        print(f"Detecting Face {video_name}" )
        ret, frame = cap.read()
        if(ret == True):
            face_image = face_recognition.face_locations(frame)
            if(len(face_image) > 0):
                break
        else:
            os.rmdir(os.path.join(outputPath+'/face'))
            os.rmdir(os.path.join(outputPath+'/flow'))
            os.rmdir(os.path.join(output_path + '/'+ video_name))
            return f"{video_name} is corrupted"


    top,right,bottom,left = face_image[0]

    boxleft = left -  30 if left-30 > 0 else left 
    boxright = right + 30 if right+30 < frame.shape[1]  else right
    boxtop = top - 30 if top-30 > 0 else top
    boxbottom = bottom + 30 if bottom+30 < frame.shape[0] else bottom

    size = (640,360)

    cv2.imwrite(
        facePath+ '/' + '0000' +'.jpg',
        cv2.resize(frame[boxtop:boxbottom,boxleft:boxright,:],size)
    )

    frameCount = 1

    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:
            try:
                face_image = face_recognition.face_locations(frame)
                if(len(face_image) > 0):
                    top,right,bottom,left = face_image[0]

                    if(top < boxtop or bottom > boxbottom or right > boxright or left < boxleft):
                        # update box
                        boxtop = top - 30;
                        boxright = right + 30
                        boxleft = left - 30
                        boxbottom = bottom + 30


                    paddedCount = '%04d' % frameCount
                    cv2.imwrite(
                        facePath+'/'+ paddedCount +'.jpg',
                        cv2.resize(frame[boxtop:boxbottom,boxleft:boxright,:],size)            
                    )

                    frameCount+=1
                    if frameCount == 120:
                        break
            except:
                print("Frame skipped")
                pass

        else:
            break
    
    return f"Processed {video} \n"


In [4]:
def preprocessing(input_path,output_path):
    files = os.listdir(input_path)
    files.sort()
    videos = []
    for file in files:
        path = os.path.join(input_path + '/'+file)
        videos.append(path)
    
    with concurrent.futures.ProcessPoolExecutor(max_workers=multiprocessing.cpu_count()) as executor:
        results = [executor.submit(extract_frames,video,output_path) for video in videos]
        
        for f in concurrent.futures.as_completed(results):
            print(f.result())
            
    print("Done Processing")

In [ ]:
#Create output directory manually
#os.mkdir('Output')
import time 

start = time.perf_counter()

input_path = "/home/sreejan774/Desktop/projects/MinorCodes/input"
output_path = "/home/sreejan774/Desktop/projects/MinorCodes/output"
preprocessing(input_path,output_path)

finish = time.perf_counter()
print(f"Finished in {round(finish-start,2)} secounds")